# **Lab 1 PySpark:** 


In these labs we will be using the "[[NeurIPS 2020] Data Science for COVID-19 (DS4C)](https://www.kaggle.com/datasets/kimjihoo/coronavirusdataset?select=PatientInfo.csv)" dataset, retrieved from [Kaggle](https://www.kaggle.com/) on 1/6/2022, for educational non commercial purpose, License
[CC BY-NC-SA 4.0
](https://creativecommons.org/licenses/by-nc-sa/4.0/)


The csv files that we will be using in this lab is **PatientInfo**.



# File 📁

##PatientInfo.csv

**patient_id**
the ID of the patient

**sex**
the sex of the patient

**age**
the age of the patient

**country**
the country of the patient

**province**
the province of the patient

**city**
the city of the patient

**infection_case**
the case of infection

**infected_by**
the ID of who infected the patient


**contact_number**
the number of contacts with people

**symptom_onset_date**
the date of symptom onset

**confirmed_date**
the date of being confirmed

**released_date**
the date of being released

**deceased_date**
the date of being deceased

**state**
isolated / released / deceased

# Installing PySpark:


Kindly install pyspark ⌨

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 56.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=dabd33b95d3de31f2c0d7afe93ee7d6ae33c112249105f90f35a638f734db29f
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


Check the pyspark version

In [5]:
!pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.2.1
      /_/
                        
Using Scala version 2.12.15, OpenJDK 64-Bit Server VM, 11.0.15
Branch HEAD
Compiled by user hgao on 2022-01-20T19:26:14Z
Revision 4f25b3f71238a00508a356591553f2dfa89f8290
Url https://github.com/apache/spark
Type --help for more information.


Importing sparksession then create a spark session

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count
import pyspark.sql.functions as fn
import numpy as np


In [20]:
spark = (SparkSession
.builder
.appName("Lab1_G2")
.getOrCreate())
sc = spark.sparkContext

# Follow the instructions above the cells below:


Kindly load the PatientInfo.csv file and show the first 5 rows

❌ PLEASE do NOT change the cell below 

In [21]:
!gdown https://drive.google.com/uc?id=1PB6wBDVTM_eocxOyi0lWlLBQOlH0rLe_ -O PatientInfo.csv

Downloading...
From: https://drive.google.com/uc?id=1PB6wBDVTM_eocxOyi0lWlLBQOlH0rLe_
To: /content/PatientInfo.csv
100% 489k/489k [00:00<00:00, 106MB/s]


load the **PatientInfo.csv** file downloaded and put it in a dataframe called **patient**

In [22]:
df = spark.read.option("header", "true").csv('/content/PatientInfo.csv')

Show the first 5 rows

In [23]:
df.show(5)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|         null|released|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|   202

Display the schema for the dataset

In [24]:
df.printSchema()

root
 |-- patient_id: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: string (nullable = true)
 |-- released_date: string (nullable = true)
 |-- deceased_date: string (nullable = true)
 |-- state: string (nullable = true)



Display the statistical summary

In [25]:
df.describe().show()

+-------+--------------------+------+----+----------+--------+--------------+--------------------+--------------------+--------------------+------------------+--------------+-------------+-------------+--------+
|summary|          patient_id|   sex| age|   country|province|          city|      infection_case|         infected_by|      contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+-------+--------------------+------+----+----------+--------+--------------+--------------------+--------------------+--------------------+------------------+--------------+-------------+-------------+--------+
|  count|                5165|  4043|3785|      5165|    5165|          5071|                4246|                1346|                 791|               690|          5162|         1587|           66|    5165|
|   mean|2.8636345618679576E9|  null|null|      null|    null|          null|                null|2.2845944015643125E9|1.6772572523506988E7|            

How many people survived, and how many didn't survive?

In [30]:
# df['infection_case'].
df.groupBy('state').agg(fn.count('state')).show()

+--------+------------+
|   state|count(state)|
+--------+------------+
|isolated|        2158|
|released|        2929|
|deceased|          78|
+--------+------------+



Display the number of null values in each column

In [31]:
df.select([count(fn.when(fn.isnan(c) | fn.col(c).isNull(), c)).alias(c) for c in df.columns] ).show()

+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+
|patient_id| sex| age|country|province|city|infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|state|
+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+
|         0|1122|1380|      0|       0|  94|           919|       3819|          4374|              4475|             3|         3578|         5099|    0|
+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+



# Data preprocessing

Kindly fill the nulls in the **deceased_date** with the **released_date**

In [61]:
from pyspark.sql.functions import coalesce
df2 =df.withColumn('deceased_date', coalesce('deceased_date', 'released_date'))


In [62]:
df2.select([count(fn.when(fn.isnan(c) | fn.col(c).isNull(), c)).alias(c) for c in df2.columns] ).show()

+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+
|patient_id| sex| age|country|province|city|infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|state|
+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+
|         0|1122|1380|      0|       0|  94|           919|       3819|          4374|              4475|             3|         3578|         3514|    0|
+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+



Drop values having a null deceased_date

In [150]:
df3 =df2.na.drop(subset=["deceased_date"])

In [151]:
df3.select([count(fn.when(fn.isnan(c) | fn.col(c).isNull(), c)).alias(c) for c in df3.columns] ).show()

+----------+---+---+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+
|patient_id|sex|age|country|province|city|infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|state|
+----------+---+---+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+
|         0|  1| 11|      0|       0|  51|           611|       1364|          1264|              1480|             0|           64|            0|    0|
+----------+---+---+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+



Add a column named no_days which is difference between the deceased_date and the confirmed_date then show the top 5 rows

Hint: You need to typecase these coumns as date first

In [152]:
df3.select(
      fn.to_date(fn.col("deceased_date")),
      fn.to_date(fn.col("confirmed_date")),
      fn.datediff(fn.col("deceased_date"),fn.col("confirmed_date")).alias("no_days")
    ).show()

+----------------------+-----------------------+-------+
|to_date(deceased_date)|to_date(confirmed_date)|no_days|
+----------------------+-----------------------+-------+
|            2020-02-05|             2020-01-23|     13|
|            2020-03-02|             2020-01-30|     32|
|            2020-02-19|             2020-01-30|     20|
|            2020-02-15|             2020-01-30|     16|
|            2020-02-24|             2020-01-31|     24|
|            2020-02-19|             2020-01-31|     19|
|            2020-02-10|             2020-01-31|     10|
|            2020-02-24|             2020-02-02|     22|
|            2020-02-21|             2020-02-05|     16|
|            2020-02-29|             2020-02-05|     24|
|            2020-02-29|             2020-02-06|     23|
|            2020-02-27|             2020-02-07|     20|
|            2020-03-12|             2020-02-16|     25|
|            2020-03-11|             2020-02-19|     21|
|            2020-03-01|       

In [153]:
df3 =df3.withColumn('no_days' , fn.datediff(  fn.to_date(fn.col("deceased_date")),  fn.to_date(fn.col("confirmed_date"))  ) )

Add a **is_male** column if male then it should yield 1, else then 0 and show 

1.   List item

1.   List item
2.   List item


2.   List item

your dataframe

In [154]:
from pyspark.sql.functions import when

In [155]:
df3 =df3.withColumn("is_male", when(df3.sex == "male",1).otherwise(0))

Add a **is_dead** column if patient state is released then make it 1, else 0 and show your dataframe

In [156]:
df3 = df3.withColumn("is_dead", when(df3.state == "released",1).otherwise(0))
df3.show()

+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+-------+
|patient_id|   sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|no_days|is_male|is_dead|
+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+-------+
|1000000001|  male|50s|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|   2020-02-05|released|     13|      1|      1|
|1000000002|  male|30s|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|   2020-03-02|released|     32|      1|      1|
|1000000003|  m

Kindly change the ages to bins from 10s, 0s, 10s, 20s,.etc to 0,10, 20 and show your dataframe

In [157]:
df3 =df3.na.drop(subset=["age"])

def get_age(age):
  age = int(age[:-1])
  return age

from pyspark.sql.types import IntegerType

my_fun = fn.udf(get_age , IntegerType())

df3=df3.withColumn('age',my_fun('age'))
df3.show()


+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+-------+
|patient_id|   sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|no_days|is_male|is_dead|
+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+-------+-------+-------+
|1000000001|  male| 50|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|   2020-02-05|released|     13|      1|      1|
|1000000002|  male| 30|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|   2020-03-02|released|     32|      1|      1|
|1000000003|  m

Repeat the same, but for the **no_days** return the numbers only and show your dataframe

Print the schema

In [158]:
df3.printSchema()

root
 |-- patient_id: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: string (nullable = true)
 |-- released_date: string (nullable = true)
 |-- deceased_date: string (nullable = true)
 |-- state: string (nullable = true)
 |-- no_days: integer (nullable = true)
 |-- is_male: integer (nullable = false)
 |-- is_dead: integer (nullable = false)



**age** and **no_days** need to be typecasted as Double

In [159]:
df3 =df3.withColumn("age",df3.age.cast('double'))
df3 =df3.withColumn("no_days",df3['no_days'].cast('double'))

reprint the schema to check:

In [160]:
df3.printSchema()

root
 |-- patient_id: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: double (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: string (nullable = true)
 |-- released_date: string (nullable = true)
 |-- deceased_date: string (nullable = true)
 |-- state: string (nullable = true)
 |-- no_days: double (nullable = true)
 |-- is_male: integer (nullable = false)
 |-- is_dead: integer (nullable = false)



Kindly drop the columns below

 ["patient_id","sex","infected_by","contact_number","released_date","state","symptom_onset_date","confirmed_date","deceased_date","country","no_days","city","infection_case"]

In [161]:
cols = ["patient_id","sex","infected_by","contact_number","released_date","state","symptom_onset_date","confirmed_date","deceased_date","country","no_days","city","infection_case"]
df4=df3.drop(*cols)

In [162]:
df4.show()

+----+--------+-------+-------+
| age|province|is_male|is_dead|
+----+--------+-------+-------+
|50.0|   Seoul|      1|      1|
|30.0|   Seoul|      1|      1|
|50.0|   Seoul|      1|      1|
|20.0|   Seoul|      1|      1|
|20.0|   Seoul|      0|      1|
|50.0|   Seoul|      0|      1|
|20.0|   Seoul|      1|      1|
|20.0|   Seoul|      1|      1|
|30.0|   Seoul|      1|      1|
|60.0|   Seoul|      0|      1|
|50.0|   Seoul|      0|      1|
|20.0|   Seoul|      1|      1|
|60.0|   Seoul|      0|      1|
|70.0|   Seoul|      1|      1|
|70.0|   Seoul|      1|      1|
|70.0|   Seoul|      0|      1|
|80.0|   Seoul|      1|      1|
|40.0|   Seoul|      1|      1|
|30.0|   Seoul|      1|      1|
|50.0|   Seoul|      1|      1|
+----+--------+-------+-------+
only showing top 20 rows



Please recount the number of nulls now

In [163]:
df4.select([count(fn.when(fn.isnan(c) | fn.col(c).isNull(), c)).alias(c) for c in df4.columns] ).show()

+---+--------+-------+-------+
|age|province|is_male|is_dead|
+---+--------+-------+-------+
|  0|       0|      0|      0|
+---+--------+-------+-------+

